In [1]:
import json
import zipfile
import os
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

In [2]:
from pathlib import Path
data_dir = Path('.')
data_dir.mkdir(exist_ok = True)
file_path = data_dir / Path('CVR1.zip')
dest_path = file_path

In [3]:
zip_1 = zipfile.ZipFile(dest_path, 'r')
with zip_1.open('CvrExport.json') as cvr_1:
    cvr_exp = pd.read_json(cvr_1)

In [4]:
cvr_exp.head()

,Version,ElectionId,Sessions
0,5.5.12.1,Bartow Nov 2020 General,"{'TabulatorId': 530, 'BatchId': 1, 'RecordId':..."
1,5.5.12.1,Bartow Nov 2020 General,"{'TabulatorId': 530, 'BatchId': 1, 'RecordId':..."
2,5.5.12.1,Bartow Nov 2020 General,"{'TabulatorId': 530, 'BatchId': 1, 'RecordId':..."
3,5.5.12.1,Bartow Nov 2020 General,"{'TabulatorId': 530, 'BatchId': 1, 'RecordId':..."
4,5.5.12.1,Bartow Nov 2020 General,"{'TabulatorId': 530, 'BatchId': 1, 'RecordId':..."


In [5]:
cvr_exp.size

152034

In [6]:
cvr_sessions = cvr_exp['Sessions']
unit_dict = cvr_sessions[0]
cvr_sessions.head()

0    {'TabulatorId': 530, 'BatchId': 1, 'RecordId':...
1    {'TabulatorId': 530, 'BatchId': 1, 'RecordId':...
2    {'TabulatorId': 530, 'BatchId': 1, 'RecordId':...
3    {'TabulatorId': 530, 'BatchId': 1, 'RecordId':...
4    {'TabulatorId': 530, 'BatchId': 1, 'RecordId':...
Name: Sessions, dtype: object

In [93]:
## Iterate through all contests for one ballot
unit_contests_df = pd.DataFrame()
for i in range(len(unit_contests)-1):
    contest_df = pd.DataFrame.from_dict(unit_contests[i], orient='index').drop('Marks').T
    if not unit_contests[i]['Marks']:
        for col in marks_df.columns:
            marks_df[col].values[:] = 0
    else:
        marks_df = pd.DataFrame.from_dict(unit_contests[i]['Marks'][0], orient='index').T
    contest_concat = pd.concat([contest_df, marks_df], axis=1, join="inner")
    unit_contests_df = unit_contests_df.append(contest_concat)
unit_contests_df.head()

,Id,ManifestationId,Undervotes,Overvotes,OutstackConditionIds,CandidateId,ManifestationId,PartyId,Rank,MarkDensity,IsAmbiguous,IsVote,OutstackConditionIds
0,1,133,0,0,[],2,766,0,1,100,False,True,[]
0,2,134,0,0,[],5,770,0,1,98,False,True,[]
0,3,135,0,0,[],17,783,0,1,100,False,True,[]
0,4,136,0,0,[],28,795,0,1,98,False,True,[]
0,5,137,0,0,[],31,799,0,1,99,False,True,[]


In [94]:
## Combine both tables for the first ballot into a multiindex for all ballot info
unit_df_complete = pd.merge(unit_df_nocontests, unit_contests_df, left_index=True, right_index=True)
unit_df_complete.head()

,TabulatorId,BatchId,RecordId,CountingGroupId,SessionType,PrecinctPortionId,BallotTypeId,IsCurrent,Id_x,PaperIndex,...,Overvotes,OutstackConditionIds_y,CandidateId,ManifestationId,PartyId,Rank,MarkDensity,IsAmbiguous,IsVote,OutstackConditionIds_y
0,530,1,23,3,ScannedVote,4,4,True,21,0,...,0,[],2,766,0,1,100,False,True,[]
0,530,1,23,3,ScannedVote,4,4,True,21,0,...,0,[],5,770,0,1,98,False,True,[]
0,530,1,23,3,ScannedVote,4,4,True,21,0,...,0,[],17,783,0,1,100,False,True,[]
0,530,1,23,3,ScannedVote,4,4,True,21,0,...,0,[],28,795,0,1,98,False,True,[]
0,530,1,23,3,ScannedVote,4,4,True,21,0,...,0,[],31,799,0,1,99,False,True,[]


In [17]:
# cvr_df_complete = pd.DataFrame()
# ## Compiled table for all ballot values
# for i in cvr_sessions:
#     ##Concat levels of index i dict keys/values
#     unit_df_first = pd.DataFrame.from_dict(i, orient='index').drop('ImageMask').drop('VotingSessionIdentifier').drop('UniqueVotingIdentifier').drop('Original').T
#     unit_df_second = pd.DataFrame.from_dict(i['Original'], orient='index').drop('Cards').T
#     unit_df_2lvl = pd.concat([unit_df_first, unit_df_second], axis=1, join="inner")
#     unit_df_third = pd.DataFrame.from_dict(i['Original']['Cards'][0], orient='index').drop('Contests').T
#     unit_df_nocontests = pd.concat([unit_df_2lvl, unit_df_third], axis=1, join="inner")
#     ## Contest list to table in unit example
#     unit_contests = i['Original']['Cards'][0]['Contests']
#     ## Iterate through all contests for one ballot
#     unit_contests_df = pd.DataFrame()
#     for j in unit_contests:
#         contest_df = pd.DataFrame.from_dict(j, orient='index').drop('Marks').T
#         if not j['Marks']:
#             for col in marks_df.columns:
#                 marks_df[col].values[:] = 0
#         else:
#             marks_df = pd.DataFrame.from_dict(j['Marks'][0], orient='index').T
#         contest_concat = pd.concat([contest_df, marks_df], axis=1, join="inner")
#         ## This table will exclude write-in mark density
#         unit_contests_df = unit_contests_df.append(contest_concat[unit_contests_df.columns])
#     cvr_df_complete = pd.merge(unit_df_nocontests, unit_contests_df, left_index=True, right_index=True)
# cvr_df_complete

# Questions

## How many people voted for both GOP US senators but made no vote at all for President?  (was it concentrated in any particular precinct?)

In [50]:
## Go to levels for the senate and presidential contests and count the numbers of votes for senators but no president
## Outstack Condition 6 is code for no vote
sen_no_pres_count = 0
index_count = 0
for i in cvr_sessions:
    if not 6 in i['Original']['Cards'][0]['Contests'][1].get('OutstackConditionIds'):
        if 6 in i['Original']['Cards'][0]['Contests'][0].get('OutstackConditionIds'):
            sen_no_pres_count += 1
    index_count += 1
sen_no_pres_count

63

## How many people voted for both GOP US senators but voted for Biden? (Any precinct concentration?)

In [65]:
gop_biden_pres_count = 0
purdue_count = 0
both_gop_count = 0
for i in cvr_sessions:
    if not 6 in i['Original']['Cards'][0]['Contests'][1].get('OutstackConditionIds'):
        if i['Original']['Cards'][0]['Contests'][1].get('Marks')[0].get('CandidateId') == 4:
            purdue_count += 1
            if not 6 in i['Original']['Cards'][0]['Contests'][2].get('OutstackConditionIds'):
                if i['Original']['Cards'][0]['Contests'][2].get('Marks')[0].get('CandidateId') in [9, 11, 24, 16, 19]:
                    both_gop_count += 1
                    if not 6 in i['Original']['Cards'][0]['Contests'][0].get('OutstackConditionIds'):
                        if i['Original']['Cards'][0]['Contests'][0].get('Marks')[0].get('CandidateId') == 2:
                            gop_biden_pres_count += 1
print(purdue_count, both_gop_count, gop_biden_pres_count)

37008 34809 241


## How many people split their vote between One GOP and one Dem US Senator?

In [69]:
purdue_count = 0
dem = 0
split_count = 0
for i in cvr_sessions:
    if not 6 in i['Original']['Cards'][0]['Contests'][1].get('OutstackConditionIds'):
        if i['Original']['Cards'][0]['Contests'][1].get('Marks')[0].get('CandidateId') == 4:
            purdue_count += 1
        elif not 6 in i['Original']['Cards'][0]['Contests'][2].get('OutstackConditionIds'):
            if i['Original']['Cards'][0]['Contests'][2].get('Marks')[0].get('CandidateId') in [26, 17, 18, 20, 25, 23]:
                split_count += 1
print(purdue_count, split_count)

37008 9009


## For ballots on which the mark density on more 3 or more votes is less than 15%, how many undervotes are there? (That is related to the faulty scanning issue. Those might be votes that are not counted. )

In [90]:
undervotes = []
for i in cvr_sessions:
    under_15_density = 0
    under_count = 0
    no_mark_category = 0
    registered_blank = 0
    zero_density = 0
    for j in i['Original']['Cards'][0]['Contests']:
        density = 0
        if j.get('Undervotes') == 1:
            under_count += 1
        if j.get('Marks'):
            density = j.get('Marks')[0].get('MarkDensity')
        if density > 0 and density < 15:
            under_15_density += 1
        if density == 0:
            zero_density += 1
        if not j.get('Marks'):
            no_mark_category += 1
        if 6 in j.get('OutstackConditionIds'):
            registered_blank += 1
    if under_15_density >= 3:
        undervotes.append([under_15_density, zero_density, registered_blank, no_mark_category, under_count])
undervotes_df = pd.DataFrame(undervotes, columns=['Under 15', 'Zero density', 'Labelled blank', 'No density listed', 'Undercount'])
undervotes_df

,Under 15,Zero density,Labelled blank,No density listed,Undercount
0,4,3,11,3,11
1,3,1,6,1,6
2,5,10,16,10,16
3,3,4,13,4,13
4,7,0,10,0,10
5,3,0,14,0,14
6,7,0,11,0,11
7,3,8,9,8,9
8,4,0,9,0,9
9,4,2,14,2,14


## How many “Ambiguous votes” were not counted?

In [92]:
ambiguous_not_counted = 0
for i in cvr_sessions:
    for j in i['Original']['Cards'][0]['Contests']:
        if j.get('Marks'):
            if j.get('Marks')[0].get('IsAmbiguous'):
                if not j.get('Marks')[0].get('IsVote'):
                    ambiguous_not_counted += 1
ambiguous_not_counted

212

## How many overvotes were recorded?

In [91]:
overvotes = 0
for i in cvr_sessions:
    for j in i['Original']['Cards'][0]['Contests']:
        if j.get('Overvotes') != 0:
            overvotes += j.get('Overvotes')
overvotes

192

## Can the votes for candidate be easily tabulated?

yes

## Is there a scanner batch number on the CVR?

yes